In [7]:
import pyspark
import pyspark.sql
import pyspark.sql.functions as sf

from pyspark.sql.window import Window

In [2]:
spark = pyspark.sql.SparkSession.Builder().getOrCreate()

# Twitter Example

In this notebook, we will work with some Twitter data. It was downloaded from *The Interet Archive* at https://archive.org/details/twitterstream. To demonstrate some use case for Spark window functions, we want to find the latest tweet for each hashtag.

## 1 Load Twitter Data

In a first step, we load the Twitter data. It is stored as JSONs, which are well supported by Spark.

In [ ]:
basedir = "s3://dimajix-training/data"

In [3]:
data = spark.read\
    .json(basedir + "/twitter-sample/00.json")

### Inspect Schema

Now let us inspect the schema. As we will see, the meta data for each tweet is really massive and the complete data model quite complex. Fortunately we are only interested in the tweet itself and the list of hashtags. Note that the hashtags are already extracted for us, so there is no need to use some custom extractor.

In [4]:
data.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- delete: struct (nullable = true)
 |    |-- status: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- id_str: string (nullable = true)
 |    |    |-- user_id: long (nullable = true)
 |    |    |-- user_id_str: string (nullable = true)
 |    |-- timestamp_ms: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |  

# 2 Reduce Schema

Since we don't want to work with the whole schema, let us select only the relevant columns. Note that this is only a simplification for us human beings. Spark itself would also only extract the required columns anyway, so there is no performance improvement here (which is a good thing, since Spark automatically optimizes performance).

Specifically we are interested in the following columns:
* `created_at` contains the date and time when the tweet was originally created
* `text` contains the full text of the tweet
* `entities.hashtags.text` contains an array of all hash tags

In [41]:
hashtags = data.select(
    data["created_at"],
    data["text"],
    data["entities.hashtags.text"].alias("hashtags_array")
)
hashtags.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)
 |-- hashtags_array: array (nullable = true)
 |    |-- element: string (containsNull = true)



## 3 Unpack Hashtags

Now the schema contains an array element with a list of all hashtags. But what we want and need is one record per hashtag with all other attributes copied into the generated records. This can be done with the Spark function `explode`. So we try again, but this time we generate a new record for every entry in the hashtag array.

In [43]:
hashtags = data.select(
    data["created_at"],
    data["text"],
    sf.explode(data["entities.hashtags.text"]).alias("hashtag")
)
hashtags.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)
 |-- hashtag: string (nullable = true)



# 4 Count Hashtag Frequency

Our primary goal is to find the latest tweet for every hashtag. But this only makes sense, if individual hashtags are present more than only once in our data set. So as a pre-analysis step, let us count the frequency of all hashtags.

In [44]:
result = hashtags.groupBy("hashtag") \
    .agg(sf.count("*").alias("count"))

result.orderBy(result["count"].desc()).limit(20).toPandas()

,hashtag,count
0,DolceAmoreSabotage,52
1,PushAwardsLizQuens,47
2,MTVHottest,19
3,ALDUBSafeZone,10
4,KCAMexico,7
5,FinDelMundo,6
6,ﷺ,6
7,เสียงสู๊ง,5
8,HypeFridayPlot7,5
9,เสียงสูง,5


# 5 Find Latest Tweet per Hashtag

Now we want to find the newest/latest tweet for every hashtag. This could be done using a self join, but using windows is much simpler and more natural. In addition to the latest tweet, we also want to have the count of every hashtag. We already did that before, but if we want to combine both data sets, this would require a join. Instead we also count using a window function.

In the first step, we simply perform the window aggregation and inspect the intermediate result

In [47]:
# First window for finding the newest hash tag. We will use the row number within the window to select the newest hash tag
rank_window = Window\
    .orderBy(hashtags["created_at"].desc())\
    .partitionBy("hashtag")

# Second window for counting the total frequency of every hash tag
count_window = Window\
    .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing) \
    .partitionBy("hashtag")

ranked_hashtags = hashtags.select(
    hashtags["created_at"],
    hashtags["text"],
    hashtags["hashtag"],
    sf.row_number().over(rank_window).alias("rank"),
    sf.count("*").over(count_window).alias("count")
)
ranked_hashtags.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)
 |-- hashtag: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- count: long (nullable = false)



### Inspect result

Now let us inspect the intermediate result. We do not want to view all records, but we want to restrict ourselves to the non-trivial cases where there are multiple tweets for a given hashtag (i.e. `count > 1`). 

Moreover we also want to sort the result
* First sort by count, descending. This ensures that the most commonly used hashtag comes first
* Then sort by hashtag in case that there are two hashtags with the same count
* Finally sort by rank
This sorting more or less gives us the windows concatenated into a new data frame.

In [52]:
result = ranked_hashtags.filter(ranked_hashtags["count"] > 1) \
    .orderBy(ranked_hashtags["count"].desc(), ranked_hashtags["hashtag"], ranked_hashtags["rank"].asc())

result.limit(10).toPandas()

,created_at,text,hashtag,rank,count
0,Fri Jul 29 08:00:53 +0000 2016,#DolceAmoreSabotage And holds us fiercely in...,DolceAmoreSabotage,1,52
1,Fri Jul 29 08:00:53 +0000 2016,#DolceAmoreSabotage And holds us fiercely in...,DolceAmoreSabotage,2,52
2,Fri Jul 29 08:00:52 +0000 2016,#DolceAmoreSabotage A special bond one canno...,DolceAmoreSabotage,3,52
3,Fri Jul 29 08:00:52 +0000 2016,#DolceAmoreSabotage It wraps us up in its co...,DolceAmoreSabotage,4,52
4,Fri Jul 29 08:00:52 +0000 2016,#DolceAmoreSabotage A special bond one canno...,DolceAmoreSabotage,5,52
5,Fri Jul 29 08:00:52 +0000 2016,#DolceAmoreSabotage And holds us fiercely in...,DolceAmoreSabotage,6,52
6,Fri Jul 29 08:00:51 +0000 2016,#DolceAmoreSabotage\n\nSomehow you never fail ...,DolceAmoreSabotage,7,52
7,Fri Jul 29 08:00:51 +0000 2016,#DolceAmoreSabotage\n\nSomehow you never fail ...,DolceAmoreSabotage,8,52
8,Fri Jul 29 08:00:47 +0000 2016,#DolceAmoreSabotage Ooh #PushAwardsLizQuens,DolceAmoreSabotage,9,52
9,Fri Jul 29 08:00:47 +0000 2016,#DolceAmoreSabotage Ooh #PushAwardsLizQuens,DolceAmoreSabotage,10,52


### Find latest Tweet

Now we only need to filter the result and select the tweets with `rank == 1`.

In [49]:
result = ranked_hashtags \
    .filter(result["rank"] == 1)  \
    .filter(result["count"] > 1)  \

result.limit(10).toPandas()

,created_at,text,hashtag,rank,count
0,Fri Jul 29 08:00:53 +0000 2016,#DolceAmoreSabotage And holds us fiercely in...,DolceAmoreSabotage,1,52
1,Fri Jul 29 08:00:41 +0000 2016,RT @osomatusan_clus: 【へそウォ】『おそ松：おそ松EXPO』詳細モーショ...,おそ松さん,1,2
2,Fri Jul 29 08:00:27 +0000 2016,RT @19gomgan: https://t.co/vixWseeXsh\n\n최신주소로...,19곰닷컴,1,2
3,Fri Jul 29 08:00:49 +0000 2016,🔥 #HypeFridayPlot7 🔥\n🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥 🔥🔥🔥🔥🔥...,HypeFridayPlot7,1,5
4,Fri Jul 29 08:00:15 +0000 2016,"Watch Uju by @fivestarr_kay, sing &amp; record...",Ujumusicvideo,1,2
5,Fri Jul 29 08:00:28 +0000 2016,معقوله يوصل الحسد ب عبدالعزيز محمد السعد العجل...,شماغ_البسام,1,2
6,Fri Jul 29 08:00:41 +0000 2016,اغتنم ساعة الإجابة يوم الجمعة #صورة #الدعاء_ال...,نشر_سيرته,1,2
7,Fri Jul 29 08:00:37 +0000 2016,RT @BTS_twt: 😴👋\n#JIMIN\n#갔다올게요 https://t.co/r...,갔다올게요,1,2
8,Fri Jul 29 08:00:37 +0000 2016,RT @BTS_twt: 😴👋\n#JIMIN\n#갔다올게요 https://t.co/r...,JIMIN,1,2
9,Fri Jul 29 08:00:45 +0000 2016,RT @OneDrecti0nFans: ''OMG THE BOYS WON THE AW...,KCAMexico,1,7
